In [1]:
import sys
sys.path.insert(0, sys.path[0].removesuffix('/src/jupyter_nb'))

import pandas as pd
from pycaret.classification import ClassificationExperiment
import src.utils as utils
import src.calcEMA as calc_utils
import src.myenv as myenv
from datetime import datetime

from itertools import combinations
import plotly.express as px

<built-in method __dir__ of sqlite3.Connection object at 0x7f58a572cb40>


In [2]:
# Variables
# ETCUSDT BTCUSDT
# symbol = 'ETHUSDT'
symbol = 'SOLUSDT'
interval = '1h'
# lightgbm  xgboost lr et rf
estimator = 'knn'
_compare_models = False

start_train_date = '2010-01-01'  # train < and test >= ;2020-10-01;2023-07-31 12:00:00;
start_test_date = '2023-07-31'  # train < and test >=

stop_loss = 1.5
# regression_times = 0  # 24 * 30 * 2  # horas
times_regression_PnL = 24
normalize = True
use_gpu = False
tune_model = False
apply_combination_features = False

In [3]:
cols = myenv.all_klines_cols.copy()
cols.remove('ignore')
data = utils.get_data(symbol=symbol, save_database=False, interval=interval, tail=-1, columns=cols, parse_dates=True, start_date=start_train_date)
data = data[data['open_time'] >= start_train_date]
data = utils.parse_type_fields(data, parse_dates=True)
data = utils.adjust_index(data)
data = utils.truncate_data_to_train(data)
data.info()
data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 28263 entries, 2020-08-11 06:00:00 to 2023-11-02 15:00:00
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     28263 non-null  datetime64[ns]
 1   open                          28263 non-null  float32       
 2   high                          28263 non-null  float32       
 3   low                           28263 non-null  float32       
 4   close                         28263 non-null  float32       
 5   volume                        28263 non-null  float32       
 6   close_time                    28263 non-null  int64         
 7   quote_asset_volume            28263 non-null  float32       
 8   number_of_trades              28263 non-null  int16         
 9   taker_buy_base_asset_volume   28263 non-null  float32       
 10  taker_buy_quote_asset_volume  28263 non-null  float32      

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol
ix_open_time,,,,,,,,,,,,
2020-08-11 06:00:00,2020-08-11 06:00:00,2.850000,3.470000,2.850000,2.951500,2.003226e+04,1597129199999,6.140623e+04,267,8525.750000,2.642837e+04,SOLUSDT
2020-08-11 07:00:00,2020-08-11 07:00:00,2.951500,3.135500,2.880000,2.922400,4.206937e+04,1597132799999,1.251929e+05,472,15341.030273,4.557193e+04,SOLUSDT
2020-08-11 08:00:00,2020-08-11 08:00:00,2.962600,3.000000,2.914400,2.960000,2.428076e+04,1597136399999,7.147020e+04,209,11965.570312,3.518372e+04,SOLUSDT
2020-08-11 09:00:00,2020-08-11 09:00:00,2.960000,2.973600,2.850000,2.854300,2.637123e+04,1597139999999,7.726005e+04,230,6206.299805,1.822488e+04,SOLUSDT
2020-08-11 10:00:00,2020-08-11 10:00:00,2.856600,2.932900,2.843300,2.897600,2.668594e+04,1597143599999,7.677592e+04,277,5229.470215,1.508981e+04,SOLUSDT
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-02 11:00:00,2023-11-02 11:00:00,43.209999,44.570000,43.080002,44.470001,6.674332e+05,1698926399999,2.934779e+07,-29986,386088.250000,1.698031e+07,SOLUSDT
2023-11-02 12:00:00,2023-11-02 12:00:00,44.470001,44.500000,43.560001,43.630001,5.479861e+05,1698929999999,2.417939e+07,26449,264224.781250,1.166117e+07,SOLUSDT
2023-11-02 13:00:00,2023-11-02 13:00:00,43.630001,43.810001,42.500000,43.320000,1.133168e+06,1698933599999,4.899664e+07,-4264,593748.062500,2.567668e+07,SOLUSDT


In [4]:
data = calc_utils.calc_RSI(data)
data.info()
data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 28263 entries, 2020-08-11 06:00:00 to 2023-11-02 15:00:00
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     28263 non-null  datetime64[ns]
 1   open                          28263 non-null  float32       
 2   high                          28263 non-null  float32       
 3   low                           28263 non-null  float32       
 4   close                         28263 non-null  float32       
 5   volume                        28263 non-null  float32       
 6   close_time                    28263 non-null  int64         
 7   quote_asset_volume            28263 non-null  float32       
 8   number_of_trades              28263 non-null  int16         
 9   taker_buy_base_asset_volume   28263 non-null  float32       
 10  taker_buy_quote_asset_volume  28263 non-null  float32      

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,rsi
ix_open_time,,,,,,,,,,,,,
2020-08-11 06:00:00,2020-08-11 06:00:00,2.850000,3.470000,2.850000,2.951500,2.003226e+04,1597129199999,6.140623e+04,267,8525.750000,2.642837e+04,SOLUSDT,NaN
2020-08-11 07:00:00,2020-08-11 07:00:00,2.951500,3.135500,2.880000,2.922400,4.206937e+04,1597132799999,1.251929e+05,472,15341.030273,4.557193e+04,SOLUSDT,NaN
2020-08-11 08:00:00,2020-08-11 08:00:00,2.962600,3.000000,2.914400,2.960000,2.428076e+04,1597136399999,7.147020e+04,209,11965.570312,3.518372e+04,SOLUSDT,NaN
2020-08-11 09:00:00,2020-08-11 09:00:00,2.960000,2.973600,2.850000,2.854300,2.637123e+04,1597139999999,7.726005e+04,230,6206.299805,1.822488e+04,SOLUSDT,NaN
2020-08-11 10:00:00,2020-08-11 10:00:00,2.856600,2.932900,2.843300,2.897600,2.668594e+04,1597143599999,7.677592e+04,277,5229.470215,1.508981e+04,SOLUSDT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-02 11:00:00,2023-11-02 11:00:00,43.209999,44.570000,43.080002,44.470001,6.674332e+05,1698926399999,2.934779e+07,-29986,386088.250000,1.698031e+07,SOLUSDT,65.869583
2023-11-02 12:00:00,2023-11-02 12:00:00,44.470001,44.500000,43.560001,43.630001,5.479861e+05,1698929999999,2.417939e+07,26449,264224.781250,1.166117e+07,SOLUSDT,59.777241
2023-11-02 13:00:00,2023-11-02 13:00:00,43.630001,43.810001,42.500000,43.320000,1.133168e+06,1698933599999,4.899664e+07,-4264,593748.062500,2.567668e+07,SOLUSDT,57.657776


In [5]:
data = calc_utils.calc_ema_periods(data, periods_of_time=[times_regression_PnL, 200])
data.info()
data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 28263 entries, 2020-08-11 06:00:00 to 2023-11-02 15:00:00
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     28263 non-null  datetime64[ns]
 1   open                          28263 non-null  float32       
 2   high                          28263 non-null  float32       
 3   low                           28263 non-null  float32       
 4   close                         28263 non-null  float32       
 5   volume                        28263 non-null  float32       
 6   close_time                    28263 non-null  int64         
 7   quote_asset_volume            28263 non-null  float32       
 8   number_of_trades              28263 non-null  int16         
 9   taker_buy_base_asset_volume   28263 non-null  float32       
 10  taker_buy_quote_asset_volume  28263 non-null  float32      

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol,rsi,ema_24p,ema_24p_diff,ema_200p,ema_200p_diff
ix_open_time,,,,,,,,,,,,,,,,,
2020-08-11 06:00:00,2020-08-11 06:00:00,2.850000,3.470000,2.850000,2.951500,2.003226e+04,1597129199999,6.140623e+04,267,8525.750000,2.642837e+04,SOLUSDT,NaN,NaN,NaN,NaN,NaN
2020-08-11 07:00:00,2020-08-11 07:00:00,2.951500,3.135500,2.880000,2.922400,4.206937e+04,1597132799999,1.251929e+05,472,15341.030273,4.557193e+04,SOLUSDT,NaN,NaN,NaN,NaN,NaN
2020-08-11 08:00:00,2020-08-11 08:00:00,2.962600,3.000000,2.914400,2.960000,2.428076e+04,1597136399999,7.147020e+04,209,11965.570312,3.518372e+04,SOLUSDT,NaN,NaN,NaN,NaN,NaN
2020-08-11 09:00:00,2020-08-11 09:00:00,2.960000,2.973600,2.850000,2.854300,2.637123e+04,1597139999999,7.726005e+04,230,6206.299805,1.822488e+04,SOLUSDT,NaN,NaN,NaN,NaN,NaN
2020-08-11 10:00:00,2020-08-11 10:00:00,2.856600,2.932900,2.843300,2.897600,2.668594e+04,1597143599999,7.677592e+04,277,5229.470215,1.508981e+04,SOLUSDT,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-02 11:00:00,2023-11-02 11:00:00,43.209999,44.570000,43.080002,44.470001,6.674332e+05,1698926399999,2.934779e+07,-29986,386088.250000,1.698031e+07,SOLUSDT,65.869583,42.085484,5.665891,35.196526,26.347702
2023-11-02 12:00:00,2023-11-02 12:00:00,44.470001,44.500000,43.560001,43.630001,5.479861e+05,1698929999999,2.417939e+07,26449,264224.781250,1.166117e+07,SOLUSDT,59.777241,42.209045,3.366472,35.280441,23.666256
2023-11-02 13:00:00,2023-11-02 13:00:00,43.630001,43.810001,42.500000,43.320000,1.133168e+06,1698933599999,4.899664e+07,-4264,593748.062500,2.567668e+07,SOLUSDT,57.657776,42.297924,2.416373,35.360435,22.509804


In [6]:
data = utils.regression_PnL(
    data=data,
    label=myenv.label,
    diff_percent=float(stop_loss),
    max_regression_profit_and_loss=int(times_regression_PnL),
    drop_na=True,
    drop_calc_cols=True,
    strategy=None)
data.dropna(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 28064 entries, 2020-08-19 13:00:00 to 2023-11-02 15:00:00
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     28064 non-null  datetime64[ns]
 1   open                          28064 non-null  float32       
 2   high                          28064 non-null  float32       
 3   low                           28064 non-null  float32       
 4   close                         28064 non-null  float32       
 5   volume                        28064 non-null  float32       
 6   close_time                    28064 non-null  int64         
 7   quote_asset_volume            28064 non-null  float32       
 8   number_of_trades              28064 non-null  int16         
 9   taker_buy_base_asset_volume   28064 non-null  float32       
 10  taker_buy_quote_asset_volume  28064 non-null  float32      

In [7]:
data.dropna(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 28064 entries, 2020-08-19 13:00:00 to 2023-11-02 15:00:00
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     28064 non-null  datetime64[ns]
 1   open                          28064 non-null  float32       
 2   high                          28064 non-null  float32       
 3   low                           28064 non-null  float32       
 4   close                         28064 non-null  float32       
 5   volume                        28064 non-null  float32       
 6   close_time                    28064 non-null  int64         
 7   quote_asset_volume            28064 non-null  float32       
 8   number_of_trades              28064 non-null  int16         
 9   taker_buy_base_asset_volume   28064 non-null  float32       
 10  taker_buy_quote_asset_volume  28064 non-null  float32      

In [8]:
perc_data_label = data[[myenv.label, 'open_time']].groupby(myenv.label).count()
perc_data_label['perc'] = perc_data_label['open_time'] / len(data)
perc_data_label

,open_time,perc
status,,
SHORT_1.5,13837,0.493052
ESTAVEL,1022,0.036417
LONG_1.5,13205,0.470532


In [9]:
train_data, test_data = utils.split_train_test_data(data)
train_data.info()
test_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 25903 entries, 2020-08-19 13:00:00 to 2023-08-04 14:00:00
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     25903 non-null  datetime64[ns]
 1   open                          25903 non-null  float32       
 2   high                          25903 non-null  float32       
 3   low                           25903 non-null  float32       
 4   close                         25903 non-null  float32       
 5   volume                        25903 non-null  float32       
 6   close_time                    25903 non-null  int64         
 7   quote_asset_volume            25903 non-null  float32       
 8   number_of_trades              25903 non-null  int16         
 9   taker_buy_base_asset_volume   25903 non-null  float32       
 10  taker_buy_quote_asset_volume  25903 non-null  float32      

In [10]:
perc_data_label = train_data[[myenv.label, 'open_time']].groupby(myenv.label).count()
perc_data_label['perc'] = perc_data_label['open_time'] / len(train_data)
perc_data_label

,open_time,perc
status,,
SHORT_1.5,12952,0.500019
ESTAVEL,659,0.025441
LONG_1.5,12292,0.474540


In [11]:
perc_data_label = test_data[[myenv.label, 'open_time']].groupby(myenv.label).count()
perc_data_label['perc'] = perc_data_label['open_time'] / len(test_data)
perc_data_label

,open_time,perc
status,,
SHORT_1.5,885,0.409533
ESTAVEL,363,0.167978
LONG_1.5,913,0.422490


In [12]:
# BTCUSDT 1h best params: close,volume,quote_asset_volume,number_of_trades,rsi
# text_numeric_features = 'volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,rsi,ema_24p,ema_200p'.split(',')
#text_numeric_features = 'close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,rsi,ema_24p,ema_200p'
#text_numeric_features = f'volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ema_{int(times_regression_PnL)}p'
#text_numeric_features = f'quote_asset_volume,number_of_trades,ema_{int(times_regression_PnL)}p,ema_200p'
text_numeric_features = f'close,taker_buy_quote_asset_volume,ema_24p'
#text_numeric_features2 = f'volume,taker_buy_base_asset_volume,ema_{int(times_regression_PnL)}p'

numeric_features = text_numeric_features.split(',')
print(f'Numeric Features: {numeric_features} - size: {len(numeric_features)}\n')

combination_numeric_features = []
if apply_combination_features:
	for size in range(1, len(numeric_features) + 1): 
		comb = map(list, combinations(numeric_features, size))	
		for c in comb:
			res = ''
			for j in c:
				res += f'{j},'
			combination_numeric_features.append(res[0:len(res) - 1])
else:
	combination_numeric_features = [text_numeric_features]

print(f'Combination Numeric Features: \n{combination_numeric_features}')

Numeric Features: ['close', 'taker_buy_quote_asset_volume', 'ema_24p'] - size: 3

Combination Numeric Features: 
['close,taker_buy_quote_asset_volume,ema_24p']


In [13]:
from datetime import datetime
import os
from imblearn.under_sampling import InstanceHardnessThreshold, RepeatedEditedNearestNeighbours, AllKNN


#iht = InstanceHardnessThreshold(estimator=KNeighborsClassifier())

# fix_imbalance_method: condensednearestneighbour, editednearestneighborus, repeatededitednearestneighbours, allknn, instancehardnessthreshold, nearmiss, neighbourhoodcleaningrule, onesidedselection, randomundersampler, tomeklinks, randomoversampler, smote, smotenc, smoten, adasyn, borderlinesmote, kmeanssmote, svmsmote, smoteenn, smotetomek.
# 'condensednearestneighbour,editednearestneighborus,repeatededitednearestneighbours,allknn,instancehardnessthreshold,nearmiss,neighbourhoodcleaningrule,onesidedselection,randomundersampler,tomeklinks,randomoversampler,smote,smotenc,smoten,adasyn,borderlinesmote,kmeanssmote,svmsmote,smoteenn,smotetomek'.split(',')
# 'smotenc,smoten,adasyn,borderlinesmote,kmeanssmote,svmsmote,smoteenn,smotetomek'.split(',')
#imbalance_list = 'editednearestneighborus,repeatededitednearestneighbours,allknn,instancehardnessthreshold,nearmiss,neighbourhoodcleaningrule,onesidedselection,randomundersampler,tomeklinks,randomoversampler,smote,smotenc,smoten,adasyn,borderlinesmote,kmeanssmote,svmsmote,smoteenn,smotetomek'.split(',')
imbalance_list = ['instancehardnessthreshold']
simulation_results_filename = f'./resultado_simulacao_{symbol}_{interval}.csv'
print(f'simulation_results_filename: {simulation_results_filename}')

df_resultado_simulacao = pd.DataFrame()
has_simulation_file = os.path.exists(simulation_results_filename)
if has_simulation_file:
  df_resultado_simulacao = pd.read_csv(simulation_results_filename, sep=';')

setup = None

for aux_numeric_features in combination_numeric_features:
  experiement = ClassificationExperiment()
  features = []
  features += ['open_time', myenv.label]
  features += aux_numeric_features.split(',')
  print(f'features: {features}')
  for fix_imbalance_method in imbalance_list:


    print(f'fix_imbalance_method: {fix_imbalance_method}')
    fim = fix_imbalance_method
    match(fix_imbalance_method):
      case 'repeatededitednearestneighbours':
        fim = RepeatedEditedNearestNeighbours(kind_sel='all', max_iter=100, n_jobs=20, n_neighbors=3, sampling_strategy='auto')
      case 'instancehardnessthreshold':
        fim = InstanceHardnessThreshold(cv=5, estimator=None, n_jobs=20, random_state=123, sampling_strategy='auto')
      case 'allknn':
        fim = AllKNN(allow_minority=False, kind_sel='all', n_jobs=20, n_neighbors=3, sampling_strategy='auto')

    setup = experiement.setup(
        data=train_data[features].copy(),
        train_size=myenv.train_size,
        target=myenv.label,
        numeric_features=aux_numeric_features.split(','),
        date_features=['open_time'],
        create_date_columns=["hour", "day", "month"],
        data_split_shuffle=False,
        data_split_stratify=False,
        fix_imbalance=True,
        fix_imbalance_method=fim,
        remove_outliers=True,
        fold_strategy='timeseries', 
        fold=3,
        session_id=123,
        normalize=normalize,
        use_gpu=False,
        verbose=True,
        n_jobs=20,
        log_experiment=False,
    )

    best = setup.create_model(estimator, n_jobs=20, return_train_score=False, verbose=False)



simulation_results_filename: ./resultado_simulacao_SOLUSDT_1h.csv
features: ['open_time', 'status', 'close', 'taker_buy_quote_asset_volume', 'ema_24p']
fix_imbalance_method: instancehardnessthreshold


,Description,Value
0,Session id,123
1,Target,status
2,Target type,Multiclass
3,Target mapping,"SHORT_1.5: 0, ESTAVEL: 1, LONG_1.5: 2"
4,Original data shape,"(25903, 5)"
5,Transformed data shape,"(8094, 7)"
6,Transformed train set shape,"(323, 7)"
7,Transformed test set shape,"(7771, 7)"
8,Numeric features,3
9,Date features,1


In [14]:
if tune_model:
	best = setup.tune_model(best)  



In [15]:
final_model = setup.finalize_model(best, model_only=True) 
final_model

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=20, n_neighbors=5, p=2,
                     weights='uniform')

In [16]:
ajusted_test_data = test_data[features].drop(columns=[myenv.label])
ajusted_test_data.info()

df_final_predict = setup.predict_model(final_model, data=ajusted_test_data, raw_score=False, verbose=False)

res_score = None
if test_data is not None:
	df_final_predict[myenv.label] = test_data[myenv.label]
	df_final_predict['_score'] = df_final_predict['prediction_label'] == df_final_predict[myenv.label]

	print(f'Score Mean: {df_final_predict["_score"].mean()}')
	print(f'Score Group: \n{df_final_predict[[myenv.label, "_score"]].groupby(myenv.label).mean()}')
	res_score = df_final_predict[[myenv.label, '_score']].groupby(myenv.label).mean().copy()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2161 entries, 2023-08-04 15:00:00 to 2023-11-02 15:00:00
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     2161 non-null   datetime64[ns]
 1   close                         2161 non-null   float32       
 2   taker_buy_quote_asset_volume  2161 non-null   float32       
 3   ema_24p                       2161 non-null   float32       
dtypes: datetime64[ns](1), float32(3)
memory usage: 59.1 KB
Score Mean: 0.4030541416011106
Score Group: 
            _score
status            
SHORT_1.5   0.766102
ESTAVEL   0.041322
LONG_1.5  0.194962


In [17]:
data_simul = utils.simule_trading_crypto2(df_final_predict, df_final_predict.index.min(), df_final_predict.index.max(), 100.0)
data_simul

260.1717483242016